<a href="https://colab.research.google.com/github/muditmathur2020/EnglishToGermanTranslation/blob/main/English_to_German_Version_5_Final_Final_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Importing Dependencies

In [ ]:
import numpy as np
import math
import re
import time # to see how long it takes in training
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import warnings
from wordcloud import WordCloud
import unicodedata
import re
import numpy as np
import os
import io
import time
import tensorflow as tf
from tqdm.notebook import tqdm
tqdm.pandas()
from tensorflow.keras import backend as K
logger = tf.get_logger()
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, LSTM, Embedding, Input, TimeDistributed, Concatenate, RepeatVector, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model, load_model, Sequential

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')
import string
import re
from unicodedata import normalize
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu
import pandas as pd
from string import punctuation
import matplotlib.pyplot as plt
from IPython.display import Markdown, display


import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model

In [ ]:
%tensorflow_version 2.x

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds # tools for the tokenizer

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Step 2: Data Preprocessing

### read files

In [ ]:

europarl_en = open('/content/drive/MyDrive/Capstone /europarl-v7_en_de.txt', encoding='utf-8')
europarl_de = open('/content/drive/MyDrive/Capstone /europarl-v7_de_en.txt', encoding='utf-8')
print("Loaded europarl dataset")

# read the content
text_en = pd.Series(europarl_en.readlines())
text_de = pd.Series(europarl_de.readlines())

print(text_en[:50])
print(text_de[:50])

Loaded europarl dataset
0                           Resumption of the session\n
1     I declare resumed the session of the European ...
2     Although, as you will have seen, the dreaded '...
3     You have requested a debate on this subject in...
4     In the meantime, I should like to observe a mi...
5      Please rise, then, for this minute' s silence.\n
6     (The House rose and observed a minute' s silen...
7               Madam President, on a point of order.\n
8     You will be aware from the press and televisio...
9     One of the people assassinated very recently i...
10    Would it be appropriate for you, Madam Preside...
11    Yes, Mr Evans, I feel an initiative of the typ...
12    If the House agrees, I shall do as Mr Evans ha...
13              Madam President, on a point of order.\n
14    I would like your advice about Rule 143 concer...
15    My question relates to something that will com...
16    The Cunha report on multiannual guidance progr...
17    It says that this 

In [ ]:
# with open("data/nonbreaking_prefix.en", mode='r', encoding="utf-8") as f:
#     non_breaking_prefix_en = f.read()

# with open("data/nonbreaking_prefix.de", mode='r', encoding="utf-8") as f:
#     non_breaking_prefix_de = f.read()
# print(non_breaking_prefix_en[:5])
# print(non_breaking_prefix_de[:5])

### Cleaning

In [ ]:
# non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
# non_breaking_prefix_en = [' ' + pref.lower() + '.' for pref in non_breaking_prefix_en]
# non_breaking_prefix_de = non_breaking_prefix_de.split("\n")
# non_breaking_prefix_de = [' ' + pref.lower() + '.' for pref in non_breaking_prefix_de]
# print(non_breaking_prefix_en[:5])
# print(non_breaking_prefix_de[:5])

In [ ]:
# for prefix in non_breaking_prefix_en:
#     text_en = text_en.replace(prefix, prefix + '###')
# text_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", text_en)
# text_en = re.sub(r"\.###", '', text_en)
# text_en = re.sub(r" +", ' ', text_en)
# text_en = text_en.replace('###', ' ')

# text_en = text_en.split("\n")

# for prefix in non_breaking_prefix_de:
#     text_de = text_de.replace(prefix, prefix + '###')
# text_de = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", text_de)
# text_de = re.sub(r"\.###", '', text_de)
# text_de = re.sub(r" +", ' ', text_de)
# text_de = text_de.replace('###', ' ')

# text_de = text_de.split("\n")

### Tokenizing

In [ ]:
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    text_en, target_vocab_size=8000)
tokenizer_de = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    text_de, target_vocab_size=8000)

In [ ]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_DE = tokenizer_de.vocab_size + 2

# we put start and end tokens as size-1 and size-2 which are the same as
# tokenizer_size and tokenizer_size+1 because the words are from [0 to ts-1]
# tokenize_en.encode(sentence) give a list then list + list + list appends them
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in text_en]
outputs = [[VOCAB_SIZE_DE-2] + tokenizer_de.encode(sentence) + [VOCAB_SIZE_DE-1]
          for sentence in text_de]

### Remove too long sentences
Why?
(1) because when we pad we will have a hugeeee ram issuie for example sentence sizes of 1,100,2 when we pad they become 100,100,100 which we would rather loose that 100 than pad all to 100
(2) takes too much time to train

In [ ]:
MAX_LENGTH = 20 # we will still have a lot of data with max len of 20

# this part, why we do it is a bit tricky, pay attention why we do it like this:
idx_to_remove = [count for count, sent in enumerate(inputs)
                 if len(sent) > MAX_LENGTH]
# we remove in reversed because of shifting issuies when we start from begining
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

# same stuff for outputs>20
idx_to_remove = [count for count, sent in enumerate(outputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

### input/output creation
1) padding  
2) batching

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000 # how much data to keep

# now we turned our data into a dataset
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

# this is something that improves the way the dataset is stored, it increases
# the speed of accessing the data which increases training speed in return:
dataset = dataset.cache()

# now we shuffle in batches
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# this increases the speed even further:
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


# Step 3: Model Building

## A - Positional Encoding (look at the formula in the paper)

In [ ]:
class PositionalEncoding(layers.Layer):

    def __init__(self):
        # this Positional Encoder we made it a child of the Layers so it has all
        # the properties that a layer has
        super(PositionalEncoding, self).__init__()

    def get_angles(self, pos, i, d_model):
        """
        :pos: (seq_len, 1) index of the word in sentence [0 to 19]
        :i: the dimensions of the embedding (glove dims 200) then-> [0 to 199]
        :d_model: the size (dimension) of the embeded (e.g. glove size 200)
        :return: (seq_len, d_model) why? we are getting the encoding of the
                every positions vs every one of the dimensions of that word
        """
        angles = 1 / np.power(10000., (2*(i//2))/np.float32(d_model))
        return pos * angles # dim: (seq_len, d_model)

    def call(self, inputs):
        # input.shape = [batch_size, multihead_size(sz=8), each word (pos), that words embedding]
        # keep in mind we DONT change the values of the input considering
        # their positions, we just get the dims from input and calculate
        # pos encoding totally seperatly and stack them at the end
        seq_length = inputs.shape.as_list()[-2] # basically the pos
        d_model = inputs.shape.as_list()[-1] # basically the embedded values
        angles = self.get_angles(np.arange(seq_length)[:, np.newaxis],
                                 np.arange(d_model)[np.newaxis, :],
                                 d_model)
        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        # we do this because it has a [batch] dimension at the begining we add
        # it. why? because inputs and the encodings need to be same dims so we
        # make newaxis which it doesnt put 0's.... it copies those same dims for
        # all the batches...
        pos_encoding = angles[np.newaxis, ...]
        # now we need to return both the inputs and their pos_encodings
        # but we have pos_encoding in np so we make them tf
        return inputs + tf.cast(pos_encoding, tf.float32)

## B - Attention

### Attention computation (see the formula in the paper)

In [ ]:
def scaled_dot_product_attention(queries, keys, values, mask):
    # Q*K will be [output_len, d_model] * [d_model, input_len] which both are 20
    # for both english and french
    # the transpose_b=True makes keys turn to keys.T
    # each of them are this dim: [batch_size, nb_proj, seq_len, d_proj]
    # so with transpose it become: [a,b,c,d] * []
    product = tf.matmul(queries, keys, transpose_b=True)
    keys_dim = tf.cast(tf.shape(keys)[-1], tf.float32) # makes the dim_num float
    scaled_product = product / tf.math.sqrt(keys_dim) # scales it (formula stuff)

    # because this mask as the paper said is optional to prevent the program
    # from seeing the feauture. why? because when we backprop then they will
    # consider the stuff in front of them so to stop this we add -1e9 to them
    # so after softmax the probabilities become 0 for them
    if mask is not None:
        scaled_product += (mask * -1e9)

    # we apply the softmax along the last axis because we want their sum to be 1
    # scaled_product = [output_len, input_len] -> softmax on input_len so
    # basically we are keeping in_len the same but finding the probs for out_len
    # so for every ins what is the prob of each of the outs
    # (e.g. ith input, the probs [0.3,0.7] of the outs)
    probs = tf.nn.softmax(scaled_product, axis=-1)

    # attention = [output_len, input_len] * [input_len, d_model] = [output_len, d_model]
    # so now we have d_model weights for each of the output words which we will
    # feed to forwards to see their prediction for each of the out_lens
    attention = tf.matmul(probs, values)

    return attention

In [ ]:
# import numpy as np
# # this is just a test for you to see what happens in this line of code to the
# # dims (which we realized from the 4 dims, it only transposed the last two and
# # did mult only on those last two because matmul considers the other dims as
# # batch size and other stuff) (tf.matmul(a, b, transpose_b=True))
# a = np.arange(24).reshape(1,2,3,4)
# a = tf.convert_to_tensor(a, np.float32)
# b = np.arange(24).reshape(1,2,3,4)
# b = tf.convert_to_tensor(b, np.float32)
# product = tf.matmul(a, b, transpose_b=True)
# print(product.shape)


### Multi-Head attention sublayer

In [ ]:
class MultiHeadAttention(layers.Layer):
    def __init__(self, nb_proj):
        """
        :nb_proj: the number of projections for the multihead
        """
        super(MultiHeadAttention, self).__init__()
        self.nb_proj = nb_proj

    # this is the same as init but it happens when we USE the object for the
    # first time, in init it was called when we CREATED the object
    def build(self, input_shape):
        self.d_model = input_shape[-1]
        # we wanna make sure they are divisible
        assert self.d_model % self.nb_proj == 0
        # we use 2 slashes to make it integer
        self.d_proj = self.d_model // self.nb_proj

        self.query_lin = layers.Dense(self.d_model)
        self.key_lin = layers.Dense(self.d_model)
        self.value_lin = layers.Dense(self.d_model)
        self.final_lin = layers.Dense(self.d_model)

    def split_proj(self, inputs, batch_size):
        """
        :inputs: [batch_size, seq_len(20), d_model(prev layer dim)]

        :return:
            dims = [batch_size, nb_proj, seq_len, d_proj]
            nb_proj here is like channels in cnn
            we basically split the d_model to nb_proj*d_proj so d_proj is
            found by doing d_model/nb_proj
        """
        new_shape = (batch_size, -1, self.nb_proj, self.d_proj)
        # here we will get: [batch_sz, seq_len, nb_proj, d_proj]

        splited_inputs = tf.reshape(inputs, shape=new_shape)

        # so we need to reshape it to: [batch_size, nb_proj, seq_len, d_proj]
        return tf.transpose(splited_inputs, perm=[0, 2, 1, 3])


    def call(self, queries, keys, values, mask):
        batch_size = tf.shape(queries)[0]

        queries = self.query_lin(queries)
        keys = self.key_lin(keys)
        values = self.value_lin(values)

        # now we split each of them to make projs
        queries = self.split_proj(queries, batch_size)
        keys = self.split_proj(keys, batch_size)
        values = self.split_proj(values, batch_size)

        # each of the q,k,v are [batch_size, nb_proj, seq_len, d_proj]
        attention = scaled_dot_product_attention(queries, keys, values, mask)

        # now we will reverse the splits we did above: reshape + concat
        attention = tf.transpose(attention, perm=[0,2,1,3])
        # we have [batch_size, seq_len, nb_proj, d_proj] so now we concat 2, 3
        concat_attention = tf.reshape(attention, shape=(batch_size, -1, self.d_model))
        outputs = self.final_lin(concat_attention)
        return outputs

## C - Encoder

In [ ]:
class EncoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout):
        """
        :FFN_units:
            feed forward networks units: the number of units for the
            feed forward which you can see in the encoder part of the
            paper (right after the attention there is a feed forward...)
        :nb_project:
            the number of projections we have (8)
        :dropout:
            the dropout rate e.g. 0.3
        """
        super(EncoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout = dropout


    # we use this because we dont have many of the vars we want
    # when we create the Encoder, so no we can get them when we use the
    # function using 'build' instead
    def build(self, input_shape):
        self.d_model = input_shape[-1]
        # we first build the object for the multi-head-attention
        self.multi_head_attention = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

        self.dense_1 = layers.Dense(units=self.FFN_units, activation="relu")
        self.dense_2 = layers.Dense(units=self.d_model, activation="relu")
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)


    def call(self, inputs, mask, training):
        """
        :mask: which we will apply in the multi-head attention
        :training:
            it is true/false which we use dropout while we train=true to stop
            the model from overfiting but we dont use it when we are just
            testing (aka. train=false)
        """
        # if you look at the architecture you see that in the encoder
        # all of the query/key/val are the same array which is the input we
        # got from the previous layer
        attention = self.multi_head_attention(inputs, inputs, inputs, mask)

        # dropout + normalization after the attention
        attention = self.dropout_1(attention, training=training)
        # we do + inputs here because in the architecture they still concat the
        # previous inputs to our resulted attention then we normalize it
        attention = self.norm_1(attention + inputs)

        # now we do the dense in our FFN:
        outputs = self.dense_1(attention)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_2(outputs)
        outputs = self.norm_2(outputs + attention)

        return outputs


In [ ]:
class Encoder(layers.Layer):

    def __init__(self,
                 nb_encoding_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 vocab_size,
                 d_model,
                 name="encoder"):
        # we put name=name here because the name is something that belongs
        # to the layers class, so we tell it to use name="encoder"
        super(Encoder, self).__init__(name=name)
        self.nb_encoding_layers = nb_encoding_layers # the number of encoders in a row
        self.d_model = d_model # the size of the output e.g. glove(200)

        # we give vocab size for it to know the maximum number used in vocab
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)
        self.enc_layers = [EncoderLayer(FFN_units, nb_proj, dropout)
                        for _ in range(self.nb_encoding_layers)]


    def call(self, inputs, mask, training):
        # look at the paper's architecture while doing these
        # embedding with maybe glove weights....
        outputs = self.embedding(inputs)
        # the reason why we did this was because of what was writtent
        # on the paper in secssion 3.4 which they said they multiplied
        # it by sqrt of d_model
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # this will give us the concat: outputs + pos_encoding
        outputs = self.pos_encoding(outputs)
        # now we do dropout before all the encoding layers
        # we give it training=bool -> so dont do dropout when training=false
        outputs = self.dropout(outputs, training)

        # now we do the EmbeddingLayer a couple of times, not just once.
        for i in range(self.nb_encoding_layers):
            # so we apply it to the (i)th encoder in each for with these params:
            outputs = self.enc_layers[i](outputs, mask, training)

        return outputs



## D - Decoder

In [ ]:
class DecoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout):
        super(DecoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout = dropout

    def build(self, input_shape):
        self.d_model = input_shape[-1]

        # MHA 1
        self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

        # MHA 2
        self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

        # FFN
        self.dense_1 = layers.Dense(units=self.FFN_units, activation='relu')
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_3 = layers.Dropout(rate=self.dropout)
        self.norm_3 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        # check the architecture in the paper to see why we do these

        # this is the 1# attention
        attention = self.multi_head_attention_1(inputs, inputs, inputs, mask_1)
        # we give it training=bool -> so dont do dropout when training=false
        attention = self.dropout_1(attention, training)
        attention = self.norm_1(attention + inputs)

        # this is the 2# attention, this is ALOT different than before one
        # pay attention to it's inputs
        attention_2 = self.multi_head_attention_2(attention,
                                                enc_outputs,
                                                enc_outputs,
                                                mask_2)
        # we give it training=bool -> so dont do dropout when training=false
        attention_2 = self.dropout_2(attention_2, training)
        attention_2 = self.norm_2(attention_2 + inputs)

        # the denses
        outputs = self.dense_1(attention_2)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_3(outputs, training)
        outputs = self.norm_3(outputs + attention_2)

        return outputs


In [ ]:
class Decoder(layers.Layer):

    def __init__(self,
                 nb_decoding_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 vocab_size,
                 d_model,
                 name="decoder"):
        super(Decoder, self).__init__(name=name)
        self.nb_decoding_layers = nb_decoding_layers # the number of encoders in a row
        self.d_model = d_model # the size of the output e.g. glove(200)

        # we give vocab size for it to know the maximum number used in vocab
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)

        self.dec_layers = [DecoderLayer(FFN_units, nb_proj, dropout)
                        for _ in range(nb_decoding_layers)]


    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        # look at the paper's architecture while doing these
        # embedding with maybe glove weights....
        outputs = self.embedding(inputs)
        # the reason why we did this was because of what was writtent
        # on the paper in secssion 3.4 which they said they multiplied
        # it by sqrt of d_model
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # this will give us the concat: outputs + pos_encoding
        outputs = self.pos_encoding(outputs)
        # now we do dropout before all the encoding layers
        # we give it training=bool -> so dont do dropout when training=false
        outputs = self.dropout(outputs, training)

        # now we do the EmbeddingLayer a couple of times, not just once.
        for i in range(self.nb_decoding_layers):
            # so we apply it to the (i)th encoder in each for with these params:
            outputs = self.dec_layers[i](outputs,
                                             enc_outputs,
                                             mask_1,
                                             mask_2,
                                             training)
        return outputs

## E - Transformer

In [ ]:
class Transformer(tf.keras.Model):

    def __init__(self,
                 vocab_size_enc,
                 vocab_size_dec,
                 d_model,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 name="transformer"):
        super(Transformer, self).__init__(name=name)

        # initing the Objects
        self.encoder = Encoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout,
                               vocab_size_enc,
                               d_model)
        self.decoder = Decoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout,
                               vocab_size_dec,
                               d_model)
        # this is at the very end after you combined the enc & dec the output
        # will of size vocab_dec
        self.last_linear = layers.Dense(units=vocab_size_dec)


    def create_padding_mask(self, seq):
        """
        :seq: [batch_size, seq_len(20)]
        :return:
        """
        # so this gives us element wise equal true/false with broadcasting on 0
        # so we just want to see which words dont exist to give it true in all
        # the batches
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
        # so now we will return that mask we made but with 2 broadcasted new
        # dimensions so it can match the input needed in attention
        # (in the next cell I made and example to see it better)
        return mask[:, tf.newaxis, tf.newaxis, :]

    def create_look_ahead_mask(self, seq):
        """
        :seq: [batch_size, seq_len(20)]
        :return:
        """
        seq_len = tf.shape(seq)[1]
        # sample of what it produces is in the cell below
        # why do we do this? because when we predict the ith word we dont see
        # words from
        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return look_ahead_mask

    def call(self, enc_inputs, dec_inputs, training):
        # combining the encoder and decoder and masks here

        # creating the mask for encoder
        enc_mask = self.create_padding_mask(enc_inputs)

        # creating the mask for decoder

        # mask #1 is for the first decoder attention which uses the
        # output, output, output as q/k/v so we get max of the 2 masks for it
        dec_mask_1 = tf.maximum(self.create_padding_mask(dec_inputs),
                                self.create_look_ahead_mask(dec_inputs))
        # mask #2 is for the second decoder attention in which we use the
        # output of the encoder as v/k so we need to do masking on the input
        # so that later when doing q*k then *v we can get a correct output
        # this is what the video said, but i belive making this None is alot                  # try making this none later
        # more correct since we dont actually use the inputs and outputs but
        # their already masked and processed outputs from previous attentions
        dec_mask_2 = self.create_padding_mask(enc_inputs)

        enc_outputs = self.encoder(enc_inputs, enc_mask, training)
        dec_outputs = self.decoder(dec_inputs,
                                   enc_outputs,
                                   dec_mask_1,
                                   dec_mask_2,
                                   training)

        outputs = self.last_linear(dec_outputs)

        return outputs

### testing masks to see their outputs

In [ ]:
def create_padding_mask(seq):
    mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(seq):
    seq_len = tf.shape(seq)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    return look_ahead_mask

In [ ]:
# sample: 1 batch of seq_len=8 ->[1,8] this will become [1,1,1,8] then
# broadcasting will happen for later stages
# mask=1 means we should delete this
seq = tf.cast([[1, 2, 3, 0, 4, 0, 0, 0]], tf.int32)

print('This padding masking as the name says shows which words exist: (later broadcasting will happen for each batch & nb_proj & d_proj')
print(create_padding_mask(seq), end='\n\n')

print("This look ahead masking as the name says shows that only for i>=j we need to keep them (=0's), so we should not see the feature indxes (i<j) (the 1's)")
print('Have in mind that mask=1 means we need to get rid of that, dont confuse it with mask=0')
print(1 - tf.linalg.band_part(tf.ones((5, 5)), -1, 0), end='\n\n')

print('Now applying both: (pay very close attention to this samples output, very important)')
print(tf.maximum(create_padding_mask(seq),
                 create_look_ahead_mask(seq)))


This padding masking as the name says shows which words exist: (later broadcasting will happen for each batch & nb_proj & d_proj
tf.Tensor([[[[0. 0. 0. 1. 0. 1. 1. 1.]]]], shape=(1, 1, 1, 8), dtype=float32)

This look ahead masking as the name says shows that only for i>=j we need to keep them (=0's), so we should not see the feature indxes (i<j) (the 1's)
Have in mind that mask=1 means we need to get rid of that, dont confuse it with mask=0
tf.Tensor(
[[0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]], shape=(5, 5), dtype=float32)

Now applying both: (pay very close attention to this samples output, very important)
tf.Tensor(
[[[[0. 1. 1. 1. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1. 1. 1. 1.]
   [0. 0. 0. 1. 1. 1. 1. 1.]
   [0. 0. 0. 1. 1. 1. 1. 1.]
   [0. 0. 0. 1. 0. 1. 1. 1.]
   [0. 0. 0. 1. 0. 1. 1. 1.]
   [0. 0. 0. 1. 0. 1. 1. 1.]
   [0. 0. 0. 1. 0. 1. 1. 1.]]]], shape=(1, 1, 8, 8), dtype=float32)


# Step 4: Training

## Parameters

In [ ]:
tf.keras.backend.clear_session()

# Hyper-parameters:
EPOCHS = 2
D_MODEL = 128 # 512 takes more time but has a lot better results
NB_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
NB_PROJ = 8 # 8
DROPOUT = 0.1 # 0.1

transformer = Transformer(vocab_size_enc=VOCAB_SIZE_EN,
                          vocab_size_dec=VOCAB_SIZE_DE,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout=DROPOUT)


## Loss

In [ ]:
# loss

# 1) example of SparseCategoricalCrossentropy:
# y_true = [1, 2]
# y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# 2) we made fromlogits=true. why? so it gives out numbers without softmax
# applied to them
# 3) we made reduction='none'. why? normally it would sum up all the losses form
# all the batches to make it just 1 number but we dont want it, we want to get
# rid of the padded losses ourselves then sum it up...
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')
def loss_function(target, pred):
    # so by using this mask we will get rid of all the losses that
    # corrispond to 0 in our target (aka. y_true)
    mask = tf.math.logical_not(tf.math.equal(target, 0)) # [326, 4, 0] -> [1, 1, 0]
    loss_ = loss_object(target, pred) # we got the loss numbers (not their softmax probabilities)

    # make sure that both loss_ and mask have the same data type so we can mult them
    mask = tf.cast(mask, dtype=loss_.dtype)
    # we make the computed losses for 0's 0
    loss_ *= mask

    # compute the mean loss and return
    return tf.reduce_mean(loss_)

# keeps track of losses during training
train_loss = tf.keras.metrics.Mean(name="train_loss")
# keeps track of accs during training
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')


## Optimizer

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(tf.cast(step, tf.float32))
        arg2 = tf.cast(step, tf.float32) * tf.cast((self.warmup_steps**-1.5), tf.float32)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)

## Checkpoints (delete your checkpoints if there are some unexpected errors when changing your data)

In [ ]:
# making a checkpoint:
checkpoint_path = "./ckpt/"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# lets check if we already have a checkpoint
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest Checkpoint Restored...")

## Epochs

In [ ]:

for epoch in range(EPOCHS):
    print("Start of epoch {}".format(epoch+1))
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    # iterate on each batch:
    for (batch_index, (enc_inputs, targets)) in enumerate(dataset):
        # we take all the target minus the last word: <s> hello friend <e>. so we get rid of <s>.
        # why? because we are trying to predict the next word each time, so at the last step
        # we are predicting <e> and we are done, so we wont need it as an input for our decoder
        dec_inputs = targets[:, :-1]
        # we shift 1 to right because tokens are: <s> hello friend <e>. so we get rid of <s>.
        # when we want to do the predictions, we wont need to predict the <s>, we start with <s>
        dec_outputs_real = targets[:, 1:]

        # this will record everything that happens when we do predictions
        with tf.GradientTape() as tape:
            # the true is for training
            predictions = transformer(enc_inputs, dec_inputs, True)
            loss = loss_function(dec_outputs_real, predictions)

        # now we get the gradients using this method using the tape
        gradients = tape.gradient(loss, transformer.trainable_variables)
        # now we apply the gradients according to our Adam optimizer
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

        # now lets add our loss to the train loss object that keeps track of the loss
        train_loss(loss)
        train_accuracy(dec_outputs_real, predictions)

        # now let's print our loss and acc from time to time.....
        if batch_index % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch_index, train_loss.result(), train_accuracy.result()))


    # at the end of each epoch we save a checkpoint
    ckpt_save_path = ckpt_manager.save()
    print("Saved checkpoint for epoch {}!".format(epoch+1))



Start of epoch 1


Epoch 1 Batch 0 Loss 6.2007 Accuracy 0.0000
Epoch 1 Batch 50 Loss 5.9827 Accuracy 0.0045
Epoch 1 Batch 100 Loss 5.9317 Accuracy 0.0272
Epoch 1 Batch 150 Loss 5.8887 Accuracy 0.0356
Epoch 1 Batch 200 Loss 5.8205 Accuracy 0.0398
Epoch 1 Batch 250 Loss 5.7410 Accuracy 0.0426
Epoch 1 Batch 300 Loss 5.6381 Accuracy 0.0485
Epoch 1 Batch 350 Loss 5.5309 Accuracy 0.0548
Epoch 1 Batch 400 Loss 5.4276 Accuracy 0.0597
Epoch 1 Batch 450 Loss 5.3287 Accuracy 0.0640
Epoch 1 Batch 500 Loss 5.2400 Accuracy 0.0680
Epoch 1 Batch 550 Loss 5.1498 Accuracy 0.0723
Epoch 1 Batch 600 Loss 5.0622 Accuracy 0.0768
Epoch 1 Batch 650 Loss 4.9809 Accuracy 0.0812
Epoch 1 Batch 700 Loss 4.9043 Accuracy 0.0856
Epoch 1 Batch 750 Loss 4.8297 Accuracy 0.0901
Epoch 1 Batch 800 Loss 4.7625 Accuracy 0.0944
Epoch 1 Batch 850 Loss 4.6971 Accuracy 0.0987
Epoch 1 Batch 900 Loss 4.6312 Accuracy 0.1028
Epoch 1 Batch 950 Loss 4.5688 Accuracy 0.1068
Epoch 1 Batch 1000 Loss 4.5110 Accuracy 0.1105
Epoch 1 Batch 1050 Loss 4.4578 Accur

# Step 5: Evaluation

In [ ]:
def evaluate(inp_sentence):
    # turn the sentence to the tokenizer_encoded format [hi, bye] -> [241, 6]
    inp_sentence = [VOCAB_SIZE_EN-2] + tokenizer_en.encode(inp_sentence) + [VOCAB_SIZE_EN-1]
    # expand dim on axis=0 to simulate the batch dimmension
    enc_input = tf.expand_dims(inp_sentence, axis=0)

    # let's make the ouput which starts with <s> and add that axis=0 for batch=0
    output = tf.expand_dims([VOCAB_SIZE_DE-2], axis=0)

    # the loop to predict the next word of output each time and output += it
    for _ in range(MAX_LENGTH):
        # we put false because we are not training so no dropout
        # predictions = [btch_sz=1, seq_len(output_so_far), vocav_sz_de(the
        # softmax values of each word, the higher the number the higher the
        # probability for that word)]
        predictions = transformer(enc_input, output, False)
        # we want to take the last word of this prediction
        prediction = predictions[:, -1:, :]
        # we do argmax to get the index of the most probable next word
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)

        # we reached the end of the sentence
        if predicted_id == VOCAB_SIZE_DE-1:
            return tf.squeeze(output, axis=0)

        # now we know add the new prediction to the last of the output
        output = tf.concat([output, predicted_id], axis=-1)

    #even if we didn't reach the end of the sentence we can't continue
    return tf.squeeze(output, axis=0)


In [ ]:
def translate(sentence):
    output = evaluate(sentence).numpy()
    # get rid of <s> and <e> if they exist
    output = [i for i in output if i < VOCAB_SIZE_DE-2]
    # decode indexes to words e.g. [241, 6] -> [hi, bye]
    predicted_sentence = tokenizer_de.decode(output)
    print("Input: {}".format(sentence))
    print("Predicted translation: {}".format(predicted_sentence))

In [ ]:
translate("This is a great day!")

Input: This is a great day!
Predicted translation: Das ist ein großer Tag für die heutigen Tagung.



In [ ]:
# showing This is a big day for today's meeting.

In [ ]:
translate("Ready for the exam")

Input: Ready for the exam
Predicted translation: Die Rele.

